In [1]:
import casadi as ca
import casadi.tools as cat
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as mpl
import networkx as nx
import sys, copy
from scipy.integrate import solve_ivp
import matlab.engine

sys.path.append('.')
sys.path.append('ocp-solving/')

from COVIDVaccinationOCP import COVIDVaccinationOCP, rk4_mob, rhs_py_total, rhs_py
from ItalySetup import ItalySetup

from utils import *

eng = matlab.engine.start_matlab()
eng.cd('geography-paper-master/', nargout=0)
eng.run('single_build.m', nargout=0)

model_size = 10

s = ItalySetup(model_size)

N = 52           # Number of control time interval
T = 52 * 7       # N * 7
n_int_steps = 10

model_days = pd.date_range(s.start_date, s.end_date, freq='1D')
freq = '1D'  # 'W-MON'
model_step = pd.date_range(s.start_date, s.end_date, freq=freq)
mobintime = s.mobility_ts.resample(freq).mean()

N = len(model_step)
T = len(model_days)

nx = 9

pnames = 1

states = ['S', 'E', 'P', 'I', 'A', 'Q', 'H', 'R', 'V']

S, E, P, I, A, Q, H, R, V = np.arange(nx)

integ_matlab = np.array(eng.eval('x'))

p , x0= get_parameters_from_matlab(eng, s, model_size, model_days, freq)

dt = T / N / n_int_steps

obj_params = {
    'scale_ell': 1e0,
    'scale_If': 1e2,
    'scale_v': 1e-6
}

Loaded Italy Setup with 10 nodes.


In [2]:
setup = s
model_params = copy.copy(p)
ind2name = setup.ind2name
ind_to_plot = setup.ind_to_plot
pop_node = setup.pop_node

M = setup.nnodes
mobility = setup.mobility

print(f'Building OCP with {M} nodes')

c = setup.mobility

scale_ell = obj_params['scale_ell']
scale_If = obj_params['scale_If']
scale_v = obj_params['scale_v']
pnum = params_to_vector(model_params)

pnum.append(scale_ell)
pnum.append(scale_If)
pnum.append(scale_v)

# ---- decision variables ---------
states = ['S', 'E', 'P', 'I', 'A', 'Q', 'H', 'R', 'V']
states = cat.struct_symSX(states)
[S, E, P, I, A, Q, H, R, V] = states[...]

controls = cat.struct_symSX(['v', 'mob'])
[v, mob] = controls[...]

covar = cat.struct_symSX(['J'])
[J] = covar[...]

params = cat.struct_symSX(list(model_params.keys()) + ['scale_ell', 'scale_If', 'scale_v'])
[deltaE, deltaP, sigma, eta, gammaI, gammaA, gammaQ, gammaH, alphaI, alphaH, zeta, r, p,
 q, betaP0, epsilonA, epsilonI, gammaV, betaratiointime, scale_ell, scale_If, scale_v] = params[...]

pop_nodeSX = ca.SX.sym('pop_node')


# The rhs is at time zero, the time is also no used in the equation so that exoplain
rhs, rhs_ell = rhs_py(0, states.cat, controls.cat, covar.cat, params.cat, mob, pop_nodeSX)
rhs = ca.veccat(*rhs)


Building OCP with 10 nodes


In [3]:
rhs_ell = ca.veccat(*rhs_ell)  # mod

In [4]:

frhs = ca.Function('frhs', [states, controls, covar, params, pop_nodeSX],
                           [rhs, scale_ell * rhs_ell[1] + scale_v * v * v])# mod

dt = T / N / n_int_steps  # length of an integration interval
# ---- dynamic constraints --------
k1, k1ell = frhs(states, controls, covar, params, pop_nodeSX)
k2, k2ell = frhs(states + dt / 2 * k1, controls, covar, params, pop_nodeSX)
k3, k3ell = frhs(states + dt / 2 * k2, controls, covar, params, pop_nodeSX)
k4, k4ell = frhs(states + dt * k3, controls, covar, params, pop_nodeSX)
x_next = states + dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
ell_next = dt / 6 * (
        k1ell + 2 * k2ell + 2 * k3ell + k4ell)  # No need for because we sum it week by week few lines below.

rk4_step = ca.Function('rk4_step', [states, controls, covar, params, pop_nodeSX], [x_next, ell_next])

In [5]:
x_ = ca.veccat(*states[...])
ell = 0.
# x_eul = x_ + dt*k1
for k in range(n_int_steps):
    x_, ell_ = rk4_step(x_, controls, covar, params, pop_nodeSX)
    ell += ell_

rk4_int = ca.Function('rk4_int', [states, ca.veccat(controls, covar, params, pop_nodeSX)], [x_, ell],
                      ['x0', 'p'], ['xf', 'qf'])

# BUG TODO Isn't this a double multiplication by the scale parameter since ell is already multiplied ?
ell = ca.Function('ell', [states, controls, covar, params, pop_nodeSX],
                  [scale_ell * ell + scale_v * v * v, scale_ell * ell,
                   scale_v * v * v])  # Very dependent on regularization factor


In [6]:
Vars = cat.struct_symMX([
    (
        cat.entry("x", struct=states, repeat=[M, N + 1]),
        cat.entry("u", struct=controls, repeat=[M, N]),
    ),
])

Params = cat.struct_symMX([
    (
        cat.entry("cov", struct=covar, repeat=[M, N]),  # TODO BUG this make the rainfall weekly !
        cat.entry("p", struct=params),
    ),
])


In [7]:
params.cat

SX([deltaE, deltaP, sigma, eta, gammaI, gammaA, gammaQ, gammaH, alphaI, alphaH, zeta, r, p, q, betaP0, epsilonA, epsilonI, gammaV, betaratiointime, scale_ell, scale_If, scale_v])

In [8]:
lbx = Vars(-np.inf)
ubx = Vars(np.inf)

f = 0
vaccines = 0
cases = 0
reg = 0
cdot_T = 0
dyn = [None] * M
spatial = [None] * M
for i in range(M):
    dyn[i] = []
    spatial[i] = []
    for k in range(N):

        [X_, ell_ik] = rk4_int(Vars['x', i, k],
                               ca.veccat(Vars['u', i, k], Params['cov', i, k], Params['p'], pop_node[i]))
        dyn[i].append(Vars['x', i, k + 1] - X_)
        ell_ik_, cases_ik, reg_ik = ell(Vars['x', i, k], Vars['u', i, k], Params['cov', i, k], Params['p'],
                                        pop_node[i])
        f += ell_ik
        cases += cases_ik
        reg += reg_ik

        mob_ik = sum(c[i, j] * Vars['x', j, k, 'S'] / (1 + Vars['x', j, k, 'S']) for j in range(M)) # TODO WRONG
        spatial[i].append(Vars[
                              'u', i, k, 'mob'] - mob_ik)  # spatial, vaccines and dyn are put in g(x), with constraints that spatial and dyn are equal to zero
        # thus imposing the dynamics and coupling.
        vaccines += Vars[
                        'u', i, k, 'v'] * T / N  # Number of vaccine spent = num of vaccine rate * 7 (number of days)

f /= T  # Average over interval for cost ^ but not terminal cost 

In [9]:
# Final time objective:
for i in range(M):
    Bi1Bi = Vars['x', i, -1, 'S'] / (1 + Vars['x', i, -1, 'S'])
    mob_i = sum(c[i, j] * Vars['x', j, -1, 'S'] / (1 + Vars['x', j, -1, 'S']) for j in range(M))
    foi = Params['p', 'sigma'] * ((1 - Params['p', 'sigma']) * Bi1Bi + Params['p', 'sigma'] * mob_i)
    # R0 = Params['p','theta']*Params['p','beta']*Vars['x',i,-1,'S']
    # R0 /= Params['p','mu_b']*( Params['p','gamma'] + Params['p','mu'] + Params['p','alpha'] )
    # f += Params['p','scale_If']*Vars['x',i,-1,'I']
    f += Params['p', 'scale_If'] * Params['p', 'sigma'] * (
            pop_node[i] - sum(Vars['x', i, -1, name] for name in ['R', 'I', 'V'])) * foi
    cdot_T += Params['p', 'scale_If'] * Params['p', 'sigma'] * (
            pop_node[i] - sum(Vars['x', i, -1, name] for name in ['R', 'I', 'V'])) * foi

g = cat.struct_MX([
    cat.entry("dyn", expr=dyn),
    cat.entry("spatial", expr=spatial),
    cat.entry("vaccines", expr=vaccines),
])

costTerms = ca.Function('costTerms', [Vars, Params], [cases, reg, cdot_T])


# This initialize
lbg = g(0)
ubg = g(0)


ubg['vaccines'] = 100000. * M
lbg['vaccines'] = -np.inf

optimize = 1
lbx['u', :, :, 'v'] = 0.
ubx['u', :, :, 'v'] = 15000 * optimize  # = 0 if we don't want to optimize
# ubx['u',:,:,'v'] = 0
ubx['u', :, :1, 'v'] = 0.

# Set initial conditions as constraints
for name in states.keys():
    for i in range(M):
        lbx['x', i, 0, name] = ubx['x', i, 0, name] = 0#ic[name][i]

# NLOP arguments:
# 'x' : variable names to optimize: here states and control
# 'p' : params that can be change after
# 'f' is the objective function of 'x', that we ought to minize
# 'g' is a function that goes lbg < g(x,p) < ubg. If you want equality constraint then ubg = lbg = the number yoiu want it to be.
nlp = {'x': Vars, 'p': Params, 'f': f, 'g': g}
nlpFun = ca.Function('nlpFun', [Vars, Params], [f, g])
nlpJac = nlpFun.factory('nlpJac', ['i0', 'i1'], ['jac:o1:i0'])



options = {}
options['ipopt'] = {}
options['ipopt']["linear_solver"] = "ma57"
# options['ipopt']["linear_solver"] = "ma86"
# options['ipopt']["linear_solver"] = "ma97"
# options['ipopt']['bound_frac'] = 1e-4
# options['ipopt']['bound_push'] = 1e-4
# options['ipopt']['slack_bound_frac'] = 1e-4
# options['ipopt']['slack_bound_push'] = 1e-6
# options['ipopt']['hessian_constant'] = 'yes'
# options['ipopt']["tol"] = 1e-8

solver = ca.nlpsol('solver', "ipopt", nlp, options)

# options = {}
# options['ls_step_factor'] = 0.5
# options['pr_tol'] = 1e-8
# options['du_tol'] = 1e-8
# options['qp_solver'] = 'nlpsol'
# options['qp_solver_options']  = {'nlpsol':'ipopt','nlpsol_options':{'print_time':False,'ipopt.print_level':0,'ipopt.hessian_constant':'yes','ipopt.linear_solver':'ma97','ipopt.tol':1e-16}}
# # options['regularization'] = 'regularize_reduced'
# options['regularization'] = 'regularize_reduced_diag'
# options['regularization_tol'] = 1e-10
# options['linesearch'] = 'filter'
# options["iteration_callback"] = plotIterates
# self.solver = SQPmethod(nlp,options)




In [10]:
init = Vars(0)
for i, name in enumerate(states.keys()):
    for k in range(N + 1):
        # init['x',:,k,name] = sol0.y[i,k]
        init['x', :, k, name] = 0# p['x0']  # self.ic[name][i]

init['u'] = 0.

arg = {}
arg['lbg'] = lbg
arg['ubg'] = ubg

arg['lbx'] = lbx
arg['ubx'] = ubx
arg['x0'] = init
arg['p'] = Params()
# for i in range(M):
#    for k in range(N):
#        arg['p']['cov',i,k] = rainfall_norm[i,k]**params(pnum)['r']
# arg['p']['cov',i,k] = np.random.random()**params(pnum)['r']

#arg['p']['p'] = pnum

In [11]:
Params()['p']

DM([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])